# Main

# Train

## Conv VAE

In [1]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import required modules
from models.conv_vae_model import ConvVae, vae_loss_function, print_num_params
from trainer.trainer_vae import VAETrainer
from dataloader.dataloader import load_fraud_data, load_config
from utils.model_saver import save_model, get_save_directory
from utils.wandb_logger import WandBLogger

# Build the config path
config_path = "configs/conv_vae/fraud_conv_vae.config"

# Load configuration
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)
conv_vae_config = config_parser["Conv_VAE"]

# Create config dictionary for WandB
config_dict = {}
for section in config_parser.sections():
    config_dict[section] = dict(config_parser[section])

# Initialize WandB logger
wandb_logger = WandBLogger(config_dict)

# Load data
config_dict = load_config(config_path)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Create model
model = ConvVae(conv_vae_config)
print_num_params(model)

# Training parameters
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vae_loss_function

# Create trainer
trainer = VAETrainer(model, dataloaders, loss_fn, optimizer)

# Create save directory
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Helper function to extract reconstruction loss
def extract_recon_loss(model, batch, device):
    model.eval()
    with torch.no_grad():
        batch = batch.to(device)
        recon, mu, logvar = model(batch)
        recon_loss = torch.nn.functional.mse_loss(recon, batch, reduction='sum').item()
    return recon_loss

# Track best validation loss
best_val_loss = float('inf')
train_losses, val_losses = [], []

try:
    # Training loop
    for epoch in range(1, num_epochs + 1):
        train_loss = trainer.train_epoch()
        val_loss = trainer.validate_epoch()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        # Log to WandB (if enabled)
        try:
            train_batch = next(iter(dataloaders['train']))
            val_batch = next(iter(dataloaders['val']))
            train_recon_loss = extract_recon_loss(model, train_batch, trainer.device)
            val_recon_loss = extract_recon_loss(model, val_batch, trainer.device)
            wandb_logger.log_epoch(epoch, train_loss, val_loss, train_recon_loss, val_recon_loss)
        except Exception as e:
            wandb_logger.log_epoch(epoch, train_loss, val_loss)
        
        # Print progress
        print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            metadata = {'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss}
            model_path = save_model(model, save_dir, 'best_model.pt', metadata)
            print(f"New best validation loss: {val_loss:.4f}")
            wandb_logger.log_model(model_path, metadata)

    # Save final model if different from best
    if val_losses[-1] > best_val_loss:
        metadata = {'epoch': num_epochs, 'train_loss': train_losses[-1], 'val_loss': val_losses[-1]}
        save_model(model, save_dir, 'final_model.pt', metadata)

    print(f"Training complete. Best validation loss: {best_val_loss:.4f}")

except KeyboardInterrupt:
    print("\nTraining interrupted by user. Saving current state...")
    if len(train_losses) > 0:
        metadata = {
            'interrupted': True,
            'epoch': len(train_losses), 
            'train_loss': train_losses[-1],
            'val_loss': val_losses[-1] if len(val_losses) > 0 else None
        }
        save_model(model, save_dir, 'interrupted_model.pt', metadata)
        print(f"Interrupted model saved to {save_dir}")

except Exception as e:
    print(f"\nTraining failed with error: {e}")
    import traceback
    traceback.print_exc()
    
finally:
    # Clean up WandB
    try:
        wandb_logger.finish()
    except:
        pass
    print("Training session complete.")

/home/alexkstern/miniconda3/envs/credit_vae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Initializing WandB run: conv-vae-fraud_20250313_165944 (Project: fraud-classification, Entity: alexkstern)


wandb: Currently logged in as: alexkstern (alexksternteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loaded configuration from configs/conv_vae/fraud_conv_vae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 62727
Models will be saved to: saved_models/conv_vae/fraud_conv_vae/20250313_165953


Epoch 1/50: Train Loss = 704.4488, Val Loss = 1022.8286
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_165953/best_model.pt
New best validation loss: 1022.8286


...

Epoch 48/50: Train Loss = 678.9618, Val Loss = 1022.5286


Epoch 49/50: Train Loss = 678.9620, Val Loss = 1022.5313


Epoch 50/50: Train Loss = 678.9610, Val Loss = 1022.5315
Model saved to saved_models/conv_vae/fraud_conv_vae/20250313_165953/final_model.pt
Training complete. Best validation loss: 1022.5251


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/recon_loss,▆▂▆▃▆▄▃▄▃▄▆▄▂▄█▃▁▁▅▃▃▃▄▁▂▂▆▅▃▃▃▇▃▅▂▅▃█▄▄
train/total_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/recon_loss,█▆▆▅▅▃▂▃▄▂▂▂▃▂▃▃▁▃▂▂▂▁▂▂▁▂▂▂▁▂▂▁▁▂▂▂▂▁▂▁
val/total_loss,█▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,50
train/recon_loss,6752.95557
train/total_loss,678.96097
val/recon_loss,11025.6543
val/total_loss,1022.53146


Training session complete.


## Conv VQ VAE

In [3]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

SEED = 42  # You can use any integer as your seed
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU setups
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import our model and trainer for VQ-VAE
from models.conv_vqvae_model import ConvVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config

# Import the model saving utility functions (or define them here)
import os
import torch
from datetime import datetime
import re
from utils.model_saver import save_model, get_save_directory
# Build the config path relative to this notebook's location
config_path = "configs/conv_vqvae/fraud_conv_vqvae.config"

# Use configparser to maintain compatibility with your model code
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)

# Extract the model configuration from the "Conv_VQVAE" section
conv_vqvae_config = config_parser["Conv_VQVAE"]

# Use load_config for the dataloader (it expects the dictionary format)
config_dict = load_config(config_path)

# Load data (this will override defaults with settings from the config's DataLoader section)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Instantiate the VQ-VAE model using the config section
model = ConvVQVAE(conv_vqvae_config)
print_num_params(model)

# Retrieve training hyperparameters from the Trainer section (including learning rate)
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define the optimizer and loss function for the VQ-VAE
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create the trainer instance (using your existing trainer class)
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create a single timestamped directory for this training run
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss for model saving
best_val_loss = float('inf')

# Train the model and save at appropriate points
train_losses, val_losses = [], []
for epoch in range(1, num_epochs + 1):
    # Use the trainer's methods to train and validate for one epoch
    train_loss = trainer.train_epoch()
    val_loss = trainer.validate_epoch()
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Print progress
    print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    
    # Prepare metadata to save with the model
    metadata = {
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_loss
    }
    
    # Save best model if validation loss improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(model, save_dir, 'best_model.pt', metadata)
        print(f"New best validation loss: {val_loss:.4f}")

# Save final model (only if it's not already the best model)
if val_losses[-1] > best_val_loss:
    save_model(model, save_dir, 'final_model.pt', {
        'epoch': num_epochs,
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1]
    })

print(f"Training complete. Best validation loss: {best_val_loss:.4f}")
print(f"Models saved to: {save_dir}")



Loaded configuration from configs/conv_vqvae/fraud_conv_vqvae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 64164
Models will be saved to: saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341


Epoch 1/100: Train Loss = 789.5891, Val Loss = 1094.8434
Model saved to saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341/best_model.pt
New best validation loss: 1094.8434


Epoch 2/100: Train Loss = 687.4258, Val Loss = 1024.3413
Model saved to saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341/best_model.pt
New best validation loss: 1024.3413


Epoch 3/100: Train Loss = 679.2138, Val Loss = 1022.5116
Model saved to saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341/best_model.pt
New best validation loss: 1022.5116


Epoch 4/100: Train Loss = 678.5124, Val Loss = 1014.5305
Model saved to saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341/best_model.pt
New best validation loss: 1014.5305


...

Epoch 96/100: Train Loss = 657.8975, Val Loss = 998.4899


Epoch 97/100: Train Loss = 657.9173, Val Loss = 998.4830


Epoch 98/100: Train Loss = 657.9011, Val Loss = 998.4700


Epoch 99/100: Train Loss = 657.8975, Val Loss = 998.5024


Epoch 100/100: Train Loss = 657.9041, Val Loss = 998.5084
Model saved to saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341/final_model.pt
Training complete. Best validation loss: 998.4244
Models saved to: saved_models/conv_vqvae/fraud_conv_vqvae/20250313_143341


## Transformer VQVAE

In [2]:
import os
import torch
import torch.optim as optim
import numpy as np
import random

SEED = 42  # You can use any integer as your seed
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # For multi-GPU setups
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Import our model and trainer for VQ-VAE
from models.transformer_vqvae_model import TransformerVQVAE, vqvae_loss_function, print_num_params
from trainer.trainer_vqvae import VQVAETrainer
from dataloader.dataloader import load_fraud_data, load_config

# Import the model saving utility functions (or define them here)
import os
import torch
from datetime import datetime
import re
from utils.model_saver import save_model, get_save_directory
# Build the config path relative to this notebook's location
config_path = "configs/transformer_vqvae/fraud_transformer_vqvae.config"

# Use configparser to maintain compatibility with your model code
import configparser
config_parser = configparser.ConfigParser()
config_parser.read(config_path)

transformer_vqvae_config = config_parser["Transformer_VQVAE"]

# Use load_config for the dataloader (it expects the dictionary format)
config_dict = load_config(config_path)

# Load data (this will override defaults with settings from the config's DataLoader section)
data = load_fraud_data(config_path=config_path)
dataloaders = data['dataloaders']
input_dim = data['input_dim']

# Instantiate the VQ-VAE model using the config section
model = TransformerVQVAE(transformer_vqvae_config)
print_num_params(model)

# Retrieve training hyperparameters from the Trainer section (including learning rate)
lr = config_parser["Trainer"].getfloat("lr")
num_epochs = config_parser["Trainer"].getint("num_epochs")

# Define the optimizer and loss function for the VQ-VAE
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = vqvae_loss_function

# Create the trainer instance (using your existing trainer class)
trainer = VQVAETrainer(model, dataloaders, loss_fn, optimizer)

# Create a single timestamped directory for this training run
save_dir = get_save_directory(config_path)
print(f"Models will be saved to: {save_dir}")

# Track best validation loss for model saving
best_val_loss = float('inf')

# Train the model and save at appropriate points
train_losses, val_losses = [], []
for epoch in range(1, num_epochs + 1):
    # Use the trainer's methods to train and validate for one epoch
    train_loss = trainer.train_epoch()
    val_loss = trainer.validate_epoch()
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    # Print progress
    print(f"Epoch {epoch}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")
    
    # Prepare metadata to save with the model
    metadata = {
        'epoch': epoch,
        'train_loss': train_loss,
        'val_loss': val_loss
    }
    
    # Save best model if validation loss improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(model, save_dir, 'best_model.pt', metadata)
        print(f"New best validation loss: {val_loss:.4f}")

# Save final model (only if it's not already the best model)
if val_losses[-1] > best_val_loss:
    save_model(model, save_dir, 'final_model.pt', {
        'epoch': num_epochs,
        'train_loss': train_losses[-1],
        'val_loss': val_losses[-1]
    })

print(f"Training complete. Best validation loss: {best_val_loss:.4f}")
print(f"Models saved to: {save_dir}")



Loaded configuration from configs/transformer_vqvae/fraud_transformer_vqvae.config
Filtered dataset to class 1: 378 samples
Normalization statistics (calculated from class 1): {'Time': {'mean': 80790.48148148147, 'std': 48332.5139872635}, 'Amount': {'mean': 133.6764814814815, 'std': 276.3532237447719}}
Filtered dataset to class 1: 378 samples
Filtered dataset to class 1: 47 samples
Filtered dataset to class 1: 48 samples
Total number of trainable parameters: 170689
Models will be saved to: saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105


Epoch 1/50: Train Loss = 654.7753, Val Loss = 984.5373
Model saved to saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105/best_model.pt
New best validation loss: 984.5373


Epoch 2/50: Train Loss = 647.1583, Val Loss = 984.1514
Model saved to saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105/best_model.pt
New best validation loss: 984.1514


Epoch 3/50: Train Loss = 646.9973, Val Loss = 984.0501
Model saved to saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105/best_model.pt
New best validation loss: 984.0501


Epoch 4/50: Train Loss = 646.9363, Val Loss = 983.9916
Model saved to saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105/best_model.pt
New best validation loss: 983.9916


...

Epoch 46/50: Train Loss = 647.6655, Val Loss = 984.9535


Epoch 47/50: Train Loss = 647.6545, Val Loss = 984.9999


Epoch 48/50: Train Loss = 647.6301, Val Loss = 984.9526


Epoch 49/50: Train Loss = 647.7078, Val Loss = 985.0489


Epoch 50/50: Train Loss = 647.6576, Val Loss = 984.8740
Model saved to saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105/final_model.pt
Training complete. Best validation loss: 983.7335
Models saved to: saved_models/transformer_vqvae/fraud_transformer_vqvae/20250313_131105
